In [2]:
import pandas as pd
import seaborn as sns
import plotly
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


In [3]:
df = pd.read_csv("data/subscriptions.csv")
email = 'Customer Email'
name = 'Customer Name'
c_id = 'Customer ID'

In [4]:
def contains_shir(row):
    # exception
    if row[email] == 'kshirjarohannaik@gmail.com':
        return False
    elif 'shir' in row[email]:
        return True
    elif 'shir' in row[name]:
        return True
    else:
        return False

In [5]:
## to detect duplicates & shirs - need to lower

In [6]:
print(f"before removing {df.shape = }")

df[email] = df[email].str.lower()
df[name] = df[name].str.lower()
shir_mask = df.apply(contains_shir, axis=1)
print(f"{shir_mask.sum() = }")
df = df[~shir_mask]
print(f"after removing {df.shape = }")



before removing df.shape = (283, 17)
shir_mask.sum() = 13
after removing df.shape = (270, 17)


In [7]:
dup_e = df[email].duplicated(keep=False)
print(f"{dup_e.sum() = }")
dup_n = df[name].duplicated(keep=False)
print(f"{dup_n.sum() = }")
dup_c = df[c_id].duplicated(keep=False)
print(f"{dup_c.sum() = }")


dup_e.sum() = 85
dup_n.sum() = 85
dup_c.sum() = 49


### for duplicated names & emails, they are 1-1 unique

In [8]:
both_dup_df = df[dup_e & dup_n][[name, email]]#.sort_values(by = identifiers[0])
print(f"{both_dup_df.shape = }")
print((both_dup_df.groupby(email)[name].nunique() == 1).all())
print((both_dup_df.groupby(name)[email].nunique() == 1).all())

both_dup_df.shape = (80, 2)
True
True


In [9]:
dup_email_only = df[dup_e & ~dup_n][[name, email]]
print(f"{dup_email_only.shape = }")
dup_name_only = df[~dup_e & dup_n][[name, email]]
print(f"{dup_name_only.shape = }")

dup_email_only.shape = (5, 2)
dup_name_only.shape = (5, 2)


In [72]:
# Wrapper Output outside the function, so it persists across calls
output_area = widgets.Output()

def review_customer(index=0):
    if index >= len(duplicated_customers_list):
        with output_area:
            clear_output(wait=True)
            print("✅ Done reviewing all duplicates.")
        return

    customer = duplicated_customers_list[index]
    df_customer = df[df["customer"] == customer][times_m]


    button = widgets.Button(description="next")

    def on_button_clicked(b):
        # Clear and move to next customer
        review_customer(index + 1)

    button.on_click(on_button_clicked)

    with output_area:
        clear_output(wait=True)
        print(f"Reviewing customer {index + 1}/{len(duplicated_customers_list)}")
        print(f"{customer = }")
        display(df_customer)

# Start the process
display(output_area)
review_customer(0)


Output()

In [10]:
dup_email_only.sort_values(by = email).to_csv("name_unique_email_duplications.csv", index = False)
dup_name_only.sort_values(by = name).to_csv("email_unique_name_duplications.csv", index = False)

In [32]:
# the customer id is redundent
print((dup_c == (dup_e & dup_c)).all())
print((dup_c == (dup_n & dup_c)).all())
df = df.drop(c_id, axis=1)

True
True


In [40]:
df['customer'] = df[email] + '-' + df[name]
# duplicated 
dup_customer = df['customer'].duplicated(keep=False)
print(f"{dup_customer.sum() =  }")
duplicated_customers_list = df[dup_customer]['customer'].unique()

dup_customer.sum() =  80


In [45]:
# Convert relevant date columns to datetime
df['Start Date (UTC)'] = pd.to_datetime(df['Start Date (UTC)'])
df['Canceled At (UTC)'] = pd.to_datetime(df['Canceled At (UTC)'])

# Extract the month and year
df['start_month'] = df['Start Date (UTC)'].dt.to_period('M')
df['cancel_month'] = df['Canceled At (UTC)'].dt.to_period('M')


times = ['Start Date (UTC)', 'Canceled At (UTC)']
times_m = ['start_month', 'cancel_month']

resolved_rows = []

In [70]:
# Wrapper Output outside the function, so it persists across calls
output_area = widgets.Output()

def review_customer(index=0):
    if index >= len(duplicated_customers_list):
        with output_area:
            clear_output(wait=True)
            print("✅ Done reviewing all duplicates.")
        return

    customer = duplicated_customers_list[index]
    df_customer = df[df["customer"] == customer][times_m]

    # Create dropdown and button inside this function
    dropdown = widgets.Dropdown(
        options=df_customer.index.tolist(),
        description='Pick index:',
        disabled=False,
    )

    button = widgets.Button(description="Confirm")

    def on_button_clicked(b):
        resolved_rows.append(df.loc[dropdown.value])
        # Clear and move to next customer
        review_customer(index + 1)

    button.on_click(on_button_clicked)

    with output_area:
        clear_output(wait=True)
        print(f"Reviewing customer {index + 1}/{len(duplicated_customers_list)}")
        print(f"{customer = }")
        display(df_customer)
        display(dropdown, button)

# Start the process
display(output_area)
review_customer(0)


Output()